# Script to Create User Profiles based on Past Item Interactions

In [1]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.4 MB/s eta 0:00:00


In [2]:
import os

In [3]:
import pickle
import numpy as np
import random
import itertools
import io
import json

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `MIDS-Capstone` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `MIDS

In [6]:
# Hugging Face Login
# Need a HuggingFace account and an access token
from huggingface_hub import notebook_login
notebook_login()

In [6]:
# Function to download the data
def load_dict_from_s3(bucket_name, file_key):
    """
    Load dictionary from S3 pickle file.
    """
    response = s3.get_object(Bucket=bucket_name, Key=file_key)
    file_content = response['Body'].read()
    data_dict = pickle.loads(file_content)

    return data_dict

In [7]:
# Function to download data in Google Colab
def load_dict_from_drive(file_path):
    """
    Load dictionary from Google Drive pickle file.
    """
    with open(file_path, 'rb') as file:
        data_dict = pickle.load(file)

    return data_dict

## Define LLM Task to create User Profiles

In [8]:
# AWS
class ChatBot:
    def __init__(self, model_name="meta-llama-3-1-8b-instruct-005734"):
        # Initialize the SageMaker runtime client
        self.sagemaker_runtime = boto3.client('sagemaker-runtime')
        self.endpoint_name = "jumpstart-dft-llama-3-1-8b-instruct-20250225-005734"
        self.model_name = model_name

    def get_response(self, message):
        try:
            payload = json.dumps({"inputs": message})
            response = self.sagemaker_runtime.invoke_endpoint(
                EndpointName=self.endpoint_name,
                ContentType='application/json',
                Body=payload
            )
            result = json.loads(response['Body'].read().decode())

            # Print raw response for debugging
            print("\n===== RAW RESPONSE FROM SAGEMAKER =====")
            print(result)
            print("========================================\n")

            # Extract and clean response
            if isinstance(result, list) and len(result) > 0:
                generated_text = result[0].get('generated_text', '').strip()
            else:
                generated_text = "Error: Unexpected response format from SageMaker."

            return generated_text

        except Exception as e:
            print(f"Error in get_response: {e}")
            return "Error: Unable to generate a response."

    def build_prompt(self, item_list, item_info_dict):
        """
        Build a user profiling prompt based on books read
        """
        content_begin = """
        Now, you are a book analyst tasked to identify a user's favorite book genres based on their past readings. I will provide you with a list of books that a user has watched in the past.
        Each book contains two pieces of information: title and genre. Based on this information, identify in the provided data the user's TOP 3 most-read genres.

        Here are some examples of what the output should look like:

        ### Example 1:
        **INPUT**
        User's Reading History:
        - Title: The Great Gatsby, Genre: Fiction
        - Title: Thinking, Fast and Slow, Genre: Psychology
        - Title: 1984, Genre: Dystopian
        - Title: Brave New World, Genre: Dystopian
        - Title: Fahrenheit 451, Genre: Dystopian
        - Title: Blink, Genre: Psychology
        - Title: The Catcher in the Rye, Genre: Fiction

        **OUTPUT**
        User favorite genres: [Dystopian], [Fiction], [Psychology]

        ---

        ### Here is the list of previously read books:
        """

        content_end = """
        Please provide the profile strictly in the following format:
        User interests: [Interest 1], [Interest 2], [Interest 3]

        Emphasize that only the most likely three genres should be provided, and strictly adhere to the above format.
        """

        # Get Data and remove dups
        unique_items = set()
        filtered_list = []

        for item_id in item_list:
            title = item_info_dict.get(item_id, {}).get('standard_title', 'Unknown Title')
            genre = item_info_dict.get(item_id, {}).get('parsed_categories', 'Unknown Genre')

            if isinstance(genre, list):
                genre = ", ".join(genre)

            item_tuple = (title, genre)
            if item_tuple not in unique_items:
                unique_items.add(item_tuple)
                filtered_list.append(f"- Title: {title}, Genre: {genre}")

        # Join unique filtered items
        content_item_list = "\n".join(filtered_list)

        return content_begin + content_item_list + content_end


    def chat(self, item_list, item_info_dict, user_Id):
        """
        Generate a user profile based on historical book interactions.
        """
        content = self.build_prompt(item_list, item_info_dict)

        # Print the raw prompt before sending it to the LLM
        print("\n===== RAW INPUT TO LLM =====")
        print(content)
        print("============================\n")

        response = self.get_response(content)

        # Print the response received from the model
        print("\n===== RAW OUTPUT FROM LLM =====")
        print(response)
        print("===============================\n")

        return {user_Id: response}

In [20]:
# Google Colab
class ChatBot:
    def __init__(self, model_name="meta-llama/Llama-3.1-8B-Instruct"):
        """
        Initialize the ChatBot with the specified model name.
        """
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        # Optimize model loading with 4-bit quantization
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16
        )

        self.model = AutoModelForCausalLM.from_pretrained(
            model_name,
            device_map="auto",
            quantization_config=bnb_config
        )

    def get_response(self, message):
        try:
          formatted_prompt = f"[INST] {message} [/INST]"
          inputs = self.tokenizer(
              formatted_prompt,
              return_tensors="pt",
              padding=True,
              truncation=True,
              max_length=1024
            ).to("cuda")

          attention_mask = inputs.attention_mask

          output = self.model.generate(
              inputs.input_ids,
              attention_mask=attention_mask,
              max_new_tokens=150,
              temperature=0.7,
              top_p=0.9,
              do_sample=True,
              pad_token_id=self.tokenizer.eos_token_id
              )

          response = self.tokenizer.decode(output[0], skip_special_tokens=True)
          print("Generated Response:", response.strip())
          return response.strip()

        except Exception as e:
          print(f"Error in get_response: {e}")
          return "Error: Unable to generate a response."

    def build_prompt(self, item_list, item_info_dict):
        """
        Build a user profiling prompt based on books read.
        """
        content_begin = """
        You are an expert book analyst. Based on the user's reading history,
        you will identify their top 3 favorite book genres.

        ---

        ### User's Reading History:
        """

        content_end = """
        Please provide the answer in this format:
        User favorite genres: [Genre 1], [Genre 2], [Genre 3]
        """

        # Remove duplicates
        unique_items = set()
        filtered_list = []

        for item_id in item_list:
            title = item_info_dict.get(item_id, {}).get('standard_title', 'Unknown Title')
            genre = item_info_dict.get(item_id, {}).get('parsed_categories', 'Unknown Genre')

            if isinstance(genre, list):
                genre = ", ".join(genre)

            item_tuple = (title, genre)
            if item_tuple not in unique_items:
                unique_items.add(item_tuple)
                filtered_list.append(f"- Title: {title}, Genre: {genre}")

        # Format the prompt
        content_item_list = "\n".join(filtered_list)
        return content_begin + content_item_list + content_end

    def chat(self, item_list, item_info_dict, user_Id):
        """
        Generate a user profile based on historical book interactions.
        """
        content = self.build_prompt(item_list, item_info_dict)
        response = self.get_response(content)

        return {user_Id: response}

## Create User Profiles

In [ ]:
# Setup Paths - AWS
s3 = boto3.client('s3')
bucket_name = 'w210recsys'

In [7]:
# #Import User-Item interactions dictionary - AWS
# file_key = 'processed/user_item_dict.pkl'
# user_item_dict = load_dict_from_drive(bucket_name, file_key)

In [9]:
# Import User-Item interactions dictionary - Colab
file_key = '/content/drive/My Drive/Capstone - Spring 2025 Personal/user_item_dict.pkl'
user_item_dict = load_dict_from_drive(file_key)

In [10]:
for i, (key, value) in enumerate(user_item_dict.items()):
    if i >= 5:
        break
    print(f"User ID: {key}, Items: {value}")

User ID: A01038432MVI9JXYTTK5T, Items: [15693, 17623, 20231, 23671, 25313, 26282, 34776, 34901, 51220, 55669, 56795, 60236]
User ID: A100NGGXRQF0AQ, Items: [141, 3330, 5426, 8736, 15511, 21557, 25655, 26605, 38825, 45116, 59372, 60614, 63574, 65047, 70451]
User ID: A100V1W0C8BWOL, Items: [21901, 24501, 28429, 29020, 30221, 32803, 36113, 10022, 47988, 58373, 64427]
User ID: A100YHBWL4TR4D, Items: [7209, 7396, 11726, 14023, 19355, 21738, 27003, 30061, 33300, 45369, 47419, 45667, 50844, 68750]
User ID: A101446I5AWY0Z, Items: [2219, 2269, 3322, 4343, 5987, 8056, 10827, 15499, 16602, 17109, 23670, 31893, 32068, 32259, 32596, 39109, 40873, 41154, 48451, 51637, 55990, 66303, 67211, 68516, 72230]


In [11]:
###### FOR TESTING ONLY ######
N = 2
sampled_keys = random.sample(list(user_item_dict.keys()), N)
sampled_dict = {key: user_item_dict[key] for key in sampled_keys}

print(sampled_dict)

{'A2YF4ZMECJVLK9': [200, 229, 706, 818, 1488, 1736, 1976, 2039, 2500, 2533, 2776, 2893, 3262, 3290, 3466, 3521, 3540, 3716, 3957, 4033, 4158, 4447, 2435, 4531, 4575, 4594, 4884, 4943, 5194, 5530, 5796, 5814, 6154, 6481, 6494, 6611, 6748, 6852, 6855, 7036, 7087, 7384, 7435, 7835, 7935, 8254, 8294, 8300, 8680, 8768, 8902, 8954, 9068, 9174, 9232, 9555, 9684, 9740, 9757, 9813, 10119, 10550, 10807, 11004, 11403, 11559, 11613, 11839, 12132, 12356, 12594, 12611, 12820, 13040, 13107, 13154, 13334, 13649, 14101, 14069, 14249, 14483, 14505, 14614, 14717, 14876, 15034, 15486, 15723, 15764, 16246, 16260, 16275, 17068, 17070, 17100, 17454, 17568, 18243, 18442, 18454, 18627, 19023, 19276, 19449, 19964, 20286, 20359, 20361, 20370, 21007, 21234, 21366, 21603, 22086, 22472, 22545, 22725, 23480, 23825, 24091, 7846, 24218, 25771, 26342, 26609, 26654, 26823, 27000, 27367, 27378, 27421, 27607, 27791, 28709, 14830, 29126, 29220, 29283, 29396, 29528, 29537, 29703, 30147, 30273, 30864, 30918, 31128, 31451, 31

In [12]:
# #Import Items (books) metadata dictionary - AWS
# file_key = 'processed/item_info_dict.pkl'
# item_info_dict = load_dict_from_s3(bucket_name, file_key)

In [13]:
# Import Items (books) metadata dictionary - Colab
file_key ='/content/drive/My Drive/Capstone - Spring 2025 Personal/item_info_dict.pkl'
item_info_dict = load_dict_from_drive(file_key)

In [14]:
for key, value in itertools.islice(item_info_dict.items(), 5):
    print(key, ":", value)

0 : {'standard_title': 'dr seuss american icon', 'parsed_categories': ['Biography & Autobiography']}
1 : {'standard_title': 'wonderful worship in smaller churches', 'parsed_categories': ['Religion']}
2 : {'standard_title': 'rising sons and daughters life among japans new young', 'parsed_categories': ['Social Science']}
3 : {'standard_title': 'muslim womens choices religious belief and social reality', 'parsed_categories': ['Religion']}
4 : {'standard_title': 'dramatica for screenwriters', 'parsed_categories': ['Reference']}


### Load the LLM model

In [21]:
# Pick a model
model_name = "meta-llama/Llama-2-7b-chat-hf"

In [22]:
# Create an instance of the ChatBot class
chatbot = ChatBot(model_name=model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
print(f"Number of unique users: {len(user_item_dict)}")
print(f"Number of unique books: {len(item_info_dict)}")

Number of unique users: 14402
Number of unique books: 72308


In [24]:
# Create dictionary to store results
result_dict = {}
counter = 0

In [25]:
# Run code through users
for user_id in sampled_dict:
    item_list = sampled_dict[user_id]

    # Get the response dictionary and update result_dict
    response_dict = chatbot.chat(item_list, item_info_dict, user_id)
    print(f"User {user_id} Response: {response_dict}")
    result_dict.update(response_dict)

    print(counter)
    counter += 1

print(counter)

Generated Response: [INST] 
        You are an expert book analyst. Based on the user's reading history, 
        you will identify their top 3 favorite book genres.

        ---
        
        ### User's Reading History:
        - Title: the fire next time, Genre: Political Science
- Title: foundation, Genre: Education
- Title: aiding and abetting a novel, Genre: Fiction
- Title: lord of light, Genre: Fiction
- Title: the martian way, Genre: Fiction
- Title: a connecticut yankee in king arthurs court, Genre: Fiction
- Title: come to me, Genre: Fiction
- Title: welcome to higby, Genre: Labor Day
- Title: rights of man, Genre: Political science
- Title: the mill on the floss, Genre: Fiction
- Title: the history of the siege of lisbon, Genre: Fiction
- Title: room with a view, Genre: Fiction
- Title: the troll garden,, Genre: Man-woman relationships
- Title: the narrative of arthur gordon pym of nantucket, Genre: Fiction
- Title: collapse how societies choose to fail or succeed, Genre:

In [26]:
result_dict

{'A2YF4ZMECJVLK9': "[INST] \n        You are an expert book analyst. Based on the user's reading history, \n        you will identify their top 3 favorite book genres.\n\n        ---\n        \n        ### User's Reading History:\n        - Title: the fire next time, Genre: Political Science\n- Title: foundation, Genre: Education\n- Title: aiding and abetting a novel, Genre: Fiction\n- Title: lord of light, Genre: Fiction\n- Title: the martian way, Genre: Fiction\n- Title: a connecticut yankee in king arthurs court, Genre: Fiction\n- Title: come to me, Genre: Fiction\n- Title: welcome to higby, Genre: Labor Day\n- Title: rights of man, Genre: Political science\n- Title: the mill on the floss, Genre: Fiction\n- Title: the history of the siege of lisbon, Genre: Fiction\n- Title: room with a view, Genre: Fiction\n- Title: the troll garden,, Genre: Man-woman relationships\n- Title: the narrative of arthur gordon pym of nantucket, Genre: Fiction\n- Title: collapse how societies choose to fa

### Save user profiles

In [ ]:
s3_key = "data/user_profile/llama/user_profile_dict.pkl"

# Serialize the dictionary into a binary stream
pickle_buffer = io.BytesIO()
pickle.dump(result_dict, pickle_buffer)
pickle_buffer.seek(0)

# Upload to S3
s3_client.upload_fileobj(pickle_buffer, bucket_name, s3_key)